Python notebook developed by:
Rodrigo Cedeno & Emanuele Capizzi
For project ...

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
import matplotlib.figure
import fiona.crs
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
import xarray as xr
import time
import yaml
import uuid
import os
from glob import iglob # data access in file manager
import rasterio
from os.path import join # same
import plotly.graph_objects as go
import rioxarray as rio

#Hide Warnings
import warnings
warnings.filterwarnings('ignore')

#Progress bar
import ipywidgets as widgets
from termcolor import colored # Print colored text

In [2]:
file_basepath = '/home/asi/datacube_conf/data/imagery/S2'
#search inside sub folders
list_files = sorted(list(iglob(join(file_basepath, '*.tif*'), recursive=True)))
#without searching inside sub folders
#list_files = sorted(list(iglob(join(files_to_move, '*.tif'), recursive=True)))
list_files

['/home/asi/datacube_conf/data/imagery/S2/S2_20230210_30m_all_bands_clip.tif',
 '/home/asi/datacube_conf/data/imagery/S2/S2_20230322_30m_all_bands_clip.tif',
 '/home/asi/datacube_conf/data/imagery/S2/S2_20230426_30m_all_bands_clip.tif',
 '/home/asi/datacube_conf/data/imagery/S2/S2_20230625_30m_all_bands_clip.tif',
 '/home/asi/datacube_conf/data/imagery/S2/S2_20230710_30m_all_bands_clip.tif',
 '/home/asi/datacube_conf/data/imagery/S2/S2_20230819_30m_all_bands_clip.tif']

# Generate product measurements list

In [3]:
f = widgets.IntProgress(min=0, max=1, description='Processing images', bar_style='info')
display(f)
p_name = "S2_Full_Bands"
text_p0 = """name: """ + p_name + """
description: Sentinel-2 imagery containing all bands
metadata_type: eo3

metadata:
    product:
        name: """ + p_name + """
        format: GeoTIFF

storage:
    crs: EPSG:32632
    dimension_order: [time, x, y]
    resolution: 
        x: 30 
        y: -30"""

file = list_files[0]
date_uuid = file[file.find('202'):file.find('202')+8]
src = rasterio.open(file) 
xmin,ymin,xmax,ymax = src.bounds
affine = src.transform
shape = src.shape
text_p2_1 = """"""
text_p2 = """"""
text_p1 = ("""

measurements:""")
""""""
for band in range(1,src.count+1):
    band_num = str(int(band))
    text_p2_1 = """
    - name: band"""+band_num+"""
      dtype: float32
      nodata: NaN
      units: ''
"""
    text_p2 = text_p2 + text_p2_1
text_p3 = text_p0 + text_p1 + text_p2
yaml_text = open('/home/asi/datacube_conf/'+p_name+'.yaml', "wt")
yaml_text.write(text_p3)
yaml_text.close()
os.system("""cd /home/asi/datacube_conf; 
datacube product add /home/asi/datacube_conf/"""+p_name+""".yaml""")
#os.system("""cd /home/asi/datacube_conf; 
#datacube product update --allow-unsafe /home/asi/datacube_conf/"""+p_name+""".yaml""")
f.value += 1
        
print(colored('All S2 products have been updated into the ODC','green'))

IntProgress(value=0, bar_style='info', description='Processing images', max=1)

Adding "S2_Full_Bands" (this might take a while) DONE
All S2 products have been updated into the ODC


## Generate and Register Dataset 

In [4]:
f = widgets.IntProgress(min=0, max=len(list_files), description='Processing images', bar_style='info')
display(f)

for file in list_files:
    date_uuid = file[file.find('202'):file.find('202')+8]
    print(date_uuid)
    src = rasterio.open(file)
    xmin,ymin,xmax,ymax = src.bounds
    affine = src.transform
    shape = src.shape
    text_p1 = ("""$schema: https://schemas.opendatacube.org/dataset

id: 20000000-0000-0000-0000-0000"""+ str(date_uuid) +"""

product:
  name: S2_Full_Bands
  href: https://sentinel.esa.int/web/sentinel/missions/sentinel-2
  format: GeoTIFF

crs: EPSG:32632

geometry:
  type: Polygon
  coordinates: [[["""+str(xmin)+""", """+str(ymin)+"""], ["""+str(xmin)+""", """+str(ymax)+"""], ["""+str(xmax)+""", """+str(ymax)+"""], ["""+str(xmax)+""", """+str(ymin)+"""], ["""+str(xmin)+""", """+str(ymin)+"""]]]

grids:
  default:
    shape: ["""+str(shape[0])+""","""+str(shape[1])+"""]
    transform: ["""+str(affine[0])+""","""+str(affine[1])+""","""+str(affine[2])+""","""+str(affine[3])+""","""+str(affine[4])+""","""+str(affine[5])+""", 0,0,1]

lineage: {}
measurements:
""")
    text_p2 = """"""
    for band in range(1,src.count+1):
        band = int(band)
        text_p2_1 = """
  band"""+str(band)+""":
    band: """+str(band)+""" 
    path: """+str(file)+""""""
        text_p2 = text_p2 + text_p2_1
    
    text_p3 = """
properties:
  odc:file_format: GeoTIFF
  datetime: """+str(date_uuid)+"""T10:30:00Z
    """
    yaml_text = open(file.replace('.tif', '.yaml'), "wt")
    yaml_text.write(text_p1+text_p2+text_p3)
    yaml_text.close()
    os.system("""cd /home/asi/datacube_conf; 
    datacube dataset add """+file.replace('.tif', '.yaml'))
    
    f.value += 1
        
print(colored('All S2 products have been imported into the ODC','green'))

IntProgress(value=0, bar_style='info', description='Processing images', max=6)

20230210
20230322
20230426
20230625
20230710
20230819
All S2 products have been imported into the ODC
